# **Desafio 05 - Construcao de pipeline para chamada de variantes e detecção de SNVs e INDELs de DNASeq - Teste Admissional - Varsomics**


O objetivo é montar o pipeline, executar o pipeline com uma amostra de exemplo e nos enviar os resultados encontrados e responder alguns
questionamentos sobre o pipeline.

**Descricao da pipeline**
----
1) Preparacao do ambiente (criacao de diretorios de saida e de analise).

2) Rodar comandos para responder as questoes 1 e 2:


In [ ]:
%%bash
# Preparando o ambiente para a analise
mkdir genome
mv hg19.* genome/

mkdir questions/

mkdir samples
mv 510-7-BRCA_S8_L001_R1_001.fastq.gz samples/
mv 510-7-BRCA_S8_L001_R2_001.fastq.gz samples/

mkdir analysis/
cd analysis
mkdir sam bam variants

# Comandos para Questao 1
# Quantidade de sequencias - R1
zcat ../samples/510-7-BRCA_S8_L001_R1_001.fastq.gz | wc -l | awk '{ print $1/4 }' > ../questions/01_numero_reads_R1.txt
# Quantidade de sequencias - R2
zcat ../samples/510-7-BRCA_S8_L001_R2_001.fastq.gz | wc -l | awk '{ print $1/4 }' > ../questions/01_numero_reads_R2.txt

# Comandos para Questao 2
# Quantos contigs em hg19.fasta?
grep -ic ">" ../genome/hg19.fasta > ../questions/02_numero_contigs_hg19.txt


3) Alinhamento dos arquivos FASTQ contra o genoma humano (hg19), usando o programa BWA:

In [ ]:
%%bash

source activate variants_analysis

# Alinhamento com BWA
bwa mem -t 2 -R "@RG\tID:amostra01\tSM:amostra01\tPL:Illumina\tLB:amostra01\tPU:unit1" ../genome/hg19.fasta ../samples/510-7-BRCA_S8_L001_R1_001.fastq.gz ../samples/510-7-BRCA_S8_L001_R2_001.fastq.gz > sam/510-7-BRCA_S8.mapped.sam 2> sam/debug_bwa_510-7-BRCA_S8.mapped.txt

conda deactivate


4) Rodar o programa SAMTOOLS para converter os arquivos SAM para BAM, ordenado por cromossomo e coordenada do genoma.

In [ ]:
%%bash
source activate variants_analysis

# Converter SAM to BAM
samtools view -S -b sam/510-7-BRCA_S8.mapped.sam > bam/510-7-BRCA_S8.mapped.bam
samtools sort bam/510-7-BRCA_S8.mapped.bam -o bam/510-7-BRCA_S8.mapped.sorted.bam
samtools index bam/510-7-BRCA_S8.mapped.sorted.bam

# Questao 3
# chr17:41197694-41197819
# Primeiro - Arquivo SAM ordenado por chr e posicao
samtools view bam/510-7-BRCA_S8.mapped.sorted.bam > sam/510-7-BRCA_S8.mapped.sorted.sam

awk '{ if($3 == "chr17"){ if($4 >= 41197694){ if($4 <= 41197819){ print $0 } } } }' sam/510-7-BRCA_S8.mapped.sorted.sam > sam/chr17.sam
wc -l sam/chr17.sam > ../questions/03_numero_reads_chr17_41197694-41197819.txt

# Questao 4
# Obter reads nao alinhados
cd ../
samtools view -b -f 4 bam/510-7-BRCA_S8.mapped.bam > bam/unmapped_510-7-BRCA_S8.bam
samtools view bam/unmapped_510-7-BRCA_S8.bam > sam/unmapped_510-7-BRCA_S8.sam
wc -l sam/unmapped_510-7-BRCA_S8.sam > ../questions/04_numero_unmapped_reads.txt

conda deactivate

5) Rodar o programa FREEBAYES para busca de variantes (SNVs, Indels, entre outras):

In [ ]:
%%bash
source activate variants_analysis

# Variantes com frequencia maior que 50% - na amostra inteira
freebayes -C 5 -F 0.5 -m 20 -q 20 --min-coverage 20 -f ../genome/hg19.fasta -p 1 bam/510-7-BRCA_S8.mapped.sorted.bam > variants/510-7-BRCA_S8.mapped_variants.vcf

conda deactivate


6) Rodar o programa SnpEff para anotar as variantes encontradas pela analise realizada pelo programa Freebayes.

In [ ]:
#source activate variants_analysis

# Anotacao das variantes com o snpEff
# Tentei rodar pelo snpEff 5.0 - pacote conda - mas deu erro no Java por "overflow" da memoria
#snpEff hg19 variants/510-7-BRCA_S8.mapped_variants.vcf > variants/510-7-BRCA_S8.mapped_variants.ann.vcf

# conda deactivate

# Segunda tentativa - instalar o programa direto no Linux Ubuntu 18.04 e deu certo!
# Anotacao das variantes com o snpEff - Caso o primeiro nao funcione
java -Xmx8g -jar /home/luciana/programas/snpEff/snpEff/snpEff.jar hg19 variants/510-7-BRCA_S8.mapped_variants.vcf > variants/510-7-BRCA_S8.mapped_variants.ann.vcf



7) Rodar novamente a pipeline para a Questao 5 - buscar por variantes especificas para o gene BRCA (associado a cancer de mama). 

As coordenadas estao presentes no arquivo BRCA.list e no BRCA.bed.

In [ ]:
%%bash
source variants_analysis

# 1 - Tentei rodar primeiramente o SnpEff com o parametro do BRCA.bed, mas nao deu certo. 
# Retornou o arquivo com todos os cromossomos, sendo que deveria ter somente chr13 e chr17.
java -Xmx8g -jar /home/luciana/programas/snpEff/snpEff/snpEff.jar hg19 variants/510-7-BRCA_S8.mapped_variants.vcf -interval ../bed/BRCA.bed > variants/BRCA_2.ann.vcf
java -Xmx8g -jar /home/luciana/programas/snpEff/snpEff/snpEff.jar hg19 variants/BRCA.vcf > variants/BRCA.ann.vcf

# Passos anteriores
# samtools view bam/510-7-BRCA_S8.mapped.sorted.bam > sam/510-7-BRCA_S8.mapped.sorted.sam

# Questao 5 - BRCA
# Arquivo BRCA.list tem as coordenadas para o gene alvo BRCA
#chr13   32890433    32973045
#chr17   41197559    41276257
# Tive que filtrar o arquivo SAM para as posicoes e tambem adicionar somente os cabecalhos dos cromossomos 13 e 17 no arquivo SAM de saida.
awk '{ if($2 == "SN:chr13" || $2 == "SN:chr17"){ print $0 } }' sam/510-7-BRCA_S8.mapped.sam > sam/BRCA.sam
grep -i "@RG" sam/510-7-BRCA_S8.mapped.sam >> sam/BRCA.sam
grep -i "@PG" sam/510-7-BRCA_S8.mapped.sam >> sam/BRCA.sam
awk '{ if($3 == "chr13"){ if($4 >= 32890433){ if($4 <= 32973045){ print $0 } } } }' sam/510-7-BRCA_S8.mapped.sorted.sam >> sam/BRCA.sam
awk '{ if($3 == "chr17"){ if($4 >= 41197559){ if($4 <= 41276257){ print $0 } } } }' sam/510-7-BRCA_S8.mapped.sorted.sam >> sam/BRCA.sam

samtools view -S -b sam/BRCA.sam > bam/BRCA.bam
samtools index bam/BRCA.bam

# Variantes com frequencia maior que 50%
freebayes -C 5 -F 0.5 -m 20 -q 20 --min-coverage 20 -f ../genome/hg19.fasta -p 1 bam/BRCA.bam > variants/BRCA.vcf

# Anotacao das variantes com o snpEff
java -Xmx8g -jar /home/artic/programas/snpEff/snpEff/snpEff.jar hg19 variants/BRCA.vcf > variants/BRCA.ann.vcf

# Busca por modificacoes
echo "HIGH" > ../questions/05_variantes_BRCA.txt
grep -ic "HIGH" variants/BRCA.ann.vcf >> ../questions/05_variantes_BRCA.txt

echo "MODERATE" >> ../questions/05_variantes_BRCA.txt
grep -ic "MODERATE" variants/BRCA.ann.vcf >> ../questions/05_variantes_BRCA.txt

echo "LOW" >> ../questions/05_variantes_BRCA.txt
grep -ic "LOW" variants/BRCA.ann.vcf >> ../questions/05_variantes_BRCA.txt

echo "MODIFIER" >> ../questions/05_variantes_BRCA.txt
grep -ic "MODIFIER" variants/BRCA.ann.vcf >> ../questions/05_variantes_BRCA.txt


No diretorio /varsomics/bioinfotest/output do GitHub tem um questionario com 5 questoes referentes `a analise.
As respostas foram encontradas assim que a pipeline foi rodada e estao logo abaixo:

    1 - Quantas sequências de DNA de paciente sequenciados temos nos arquivos de fastqs R1 e R2 respectivamente ?
    Resposta: Temos 64276 reads sequenciados no arquivo fastq R1 e 64276 reads no R2.


    2 - Sobre o genoma humano hg19, quantos contigs tem o nosso genoma hg19 (hg19.fasta) aqui disponibilizado para este pipeline ?
    Resposta: 93 contigs (sendo 25 contigs dos cromossomos 1 ao 22, MT, X e Y).


    3 - Quantos alinhamentos há na região chr17:41197694-41197819 ?
    Resposta: 978 alinhamentos.

    4 - Quantos alinhamentos não conseguiram ser mapeados (unmapped alignments ?)
    Resposta: 2663 alinhamentos nao conseguiram ser mapeados.


    5 - Realize o alinhamento das sequências FASTQ contra o genoma de referência hg19 aqui disponibilizado, 
    e realize a chamada de variantes utilizando a região alvo BRCA.list (interesse apenas na região dos genes BRCA1 e BRCA2).  
    Realize a anotação de dados funcionais usando o SNPEFF.
    Com este arquivo em mãos , responda as seguintes perguntas ?

    5.1- Quantas variantes existem com impacto funcional (Annotation_Impact) do tipo HIGH, MODERATE, LOW ?
    Resposta:
    - HIGH - 0
    - MODERATE - 2
    - LOW - 2
    - MODIFIER - 6

    Existe alguma variante em HIGH ? Qual é cromossomo, posição e a alteração ?
    Nao apareceu variante em HIGH neste dataset. Tive que rodar o SnpEff em uma versao mais antiga (SnpEff  4.3t - 2017-11-2), 
    pois deu erro de "estouro"("overflow") de memoria no Java na versao 5.0.